# Edinburgh Living Landscape Pollinator Pledge

This notebook illustrates how to plot the location data collected by the [Edinburgh Pollinator Pledge](https://edinburghlivinglandscape.org.uk/pollinatorpledge/) initiative.

It uses the [folium](https://github.com/python-visualization/folium) library, which is a Python interface to [leaflet.js](https://leafletjs.com).

## Preliminary Steps

We start off by importing a few libraries and by reading the data into a [pandas](https://pandas.pydata.org) `DataFrame`.

In [1]:
import folium
import geopandas as gpd
import pandas as pd

from shapely.geometry import Point

In [2]:
fn = '../data/interim/swt_pollinator_data-2019-02-15.csv'
pollinator_data = pd.read_csv(fn, parse_dates=['Entry Date'])
pollinator_data = pollinator_data.rename(columns={'What type of space do you have?': 'type'})
pollinator_data.head()

,latitude,longitude,type,Plant for pollinators,Make space for nature,Expand the network,What is your first step going to be?,Entry Id,Entry Date
0,55.938051,-3.217624,Communal greenspace,NaN,Make space for nature,NaN,NaN,767,2019-12-02 13:43:00
1,55.925941,-3.277307,Small garden,Plant for pollinators,Make space for nature,NaN,Create an insect house,760,2019-11-02 07:36:00
2,55.943268,-3.288348,Large garden,Plant for pollinators,Make space for nature,NaN,"Plant native flowers, make wildlife pond",756,2019-10-02 13:09:00
3,55.956412,-3.290882,Small garden,Plant for pollinators,Make space for nature,NaN,Planting flowers,750,2019-09-02 09:10:00
4,55.899452,-3.218028,Small garden,Plant for pollinators,NaN,NaN,Decide on types of pollinators,746,2019-08-02 07:57:00


## Version 1: Import location using CircleMarkers

This approach pulls the data from the `DataFrame` directly.

We will ignore all columns apart from the first three &mdash; this is just for cosmetic reasons, we could skip this step.

In [3]:
df = pollinator_data[['latitude', 'longitude', 'type']]
df = df.dropna(subset=['latitude', 'longitude']) # drop any rows that have missing geo-coordinates
df.shape # rows x columns

(226, 3)

In [4]:
df.head()

,latitude,longitude,type
0,55.938051,-3.217624,Communal greenspace
1,55.925941,-3.277307,Small garden
2,55.943268,-3.288348,Large garden
3,55.956412,-3.290882,Small garden
4,55.899452,-3.218028,Small garden


To make it easier to import data into a map, we will create a list of triples with the data we need.

In [5]:
locations = list(zip(df.latitude, df.longitude, df.type))
locations[:5]

[(55.93805129999999, -3.2176237000000003, 'Communal greenspace'),
 (55.9259405, -3.2773065000000003, 'Small garden'),
 (55.943267500000005, -3.2883483, 'Large garden'),
 (55.9564125, -3.290882, 'Small garden'),
 (55.899451899999995, -3.2180276, 'Small garden')]

In [6]:
tiles = "openstreetmap"
edinburgh_centre = (55.953251, -3.188267)

m = folium.Map(location=edinburgh_centre, tiles=tiles, zoom_start=12)

for loc in locations:
    point = [loc[0], loc[1]]
    folium.CircleMarker(location=point,                      
                        radius = 5,
                        popup= loc[2],
                        color = 'red',
                        weight = 1,
                        fill='true',
                        fill_color='red',
                        fill_opacity=0.25).add_to(m)

m.save('pollinators_circlmarkers.html')
m

## Version 2: Import locations as GeoJSON 

This approach uses [GeoPandas](https://geopandas.readthedocs.io/en/latest/) to help organise the data as GeoJSON.

We start off by creating a list of shapely `Point` objects.

In [7]:
points = [Point(x, y) for x, y in zip(df.longitude, df.latitude)]

polli_gdf = gpd.GeoDataFrame(df, geometry=points) # create a GeoDataFrame
polli_gdf.crs = {"init": "epsg:4326"} # set a the Coordinate Reference System
polli_gdf.shape

(226, 4)

In [8]:
tiles = "openstreetmap"
edinburgh_centre = (55.953251, -3.188267)

m = folium.Map(location=edinburgh_centre, tiles=tiles, zoom_start=12)

style_function = lambda x: {"fillColor": "#00FFFFFF", "color": "#000000"}

polli_geo = folium.GeoJson(
    polli_gdf,
    tooltip=folium.GeoJsonTooltip(
        fields=["type"], 
        labels=True,
        sticky=False,
    ),
    style_function=style_function
)

m.add_child(polli_geo)

m.save("pollinators_markers.html")
m